In [ ]:
%reset -f
%load_ext rpy2.ipython

In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [365]:
# a) imports methods
import pandas as pd
import numpy as np
import seaborn as sns
import math
import random
from IPython.core.display import HTML
from deps.feature_importance import *
from deps.pipeline_funcs import *
from matplotlib import pyplot as plt
from sklearn.impute import KNNImputer
from collections import defaultdict
from collections import OrderedDict

SEED_GEN = 1000

# b) inicialization
random.seed(SEED_GEN)


# c) *import dataset
base = # add the path of the folder containing the datasets
metabolites_values_file = # add the metabolomics values file name 
df_ctrl_cv = pd.read_excel(f'{base}{metabolites_values_file}.xlsx', index_col=0)

metabolites_info_file = # add the metabolomics information file name 
                        # in this dataset, each row index in the metabolite name

df_met_class = pd.read_excel(f'{base}{metabolites_info_file}.xlsx', index_col=0)

# Step 1: Metabolites preselection

## Step 1 a) More than 20% of missing values among cases and controls

In [378]:
y_var_cat = # name of the variable differentiating cases (1)/ controls (0)

met_names = # list with name of the columns of the metabolites

df_ctrl = df_ctrl_cv[df_ctrl_cv[y_var_cat]==0]
df_cv = df_ctrl_cv[df_ctrl_cv[y_var_cat]==1]

na_df_ctrl = pd.DataFrame(df_ctrl.isna().sum())
na_df_cv = pd.DataFrame(df_cv.isna().sum())

na_df_ctrl.columns = ['ctrl']
na_df_cv.columns = ['cv']

na_df = pd.concat([na_df_ctrl, na_df_cv,], axis=1)

In [ ]:
vars_to_remove = remove_missing(na_df, df_ctrl, df_cv, 'ctrl', 'cv')
met_names = remove_vars(vars_to_remove, met_names, df_met_class)

df_ctrl_cv_pre_deletion = df_ctrl_cv[met_names].copy()

## Step 1 b) Metabolites very strongly correlated to multiple metabolites


### Pearson

In [ ]:
vars_to_remove = remove_multiple_correlated(df_ctrl_cv, met_names, 0.9, "pearson")
met_names = remove_vars(vars_to_remove, met_names, df_met_class)

### Spearman

In [ ]:
vars_to_remove = remove_multiple_correlated(df_ctrl_cv, met_names, 0.9, "spearman")
met_names = remove_vars(vars_to_remove, met_names, df_met_class)

## Step 1 C) Keeping one metabolite in pairwise very strongly correlation


### Pearson 

In [ ]:
vars_to_remove, pairs_to_choose = remove_multiple_correlated(df_ctrl_cv, met_names, 0.9, "pearson", pairs_also=True)

# observe list and exclude one of the pairs in the folloing list

In [ ]:
vars_to_exclude_chosen = [
    "matabolite_a",     
                         ]
met_names = remove_vars(vars_to_exclude_chosen, met_names, df_met_class)

### Spearman

In [ ]:
vars_to_remove, pairs_to_choose = remove_multiple_correlated(df_ctrl_cv, met_names, 0.9, "spearman", pairs_also=True)
# observe list and exclude one of the pairs in the following list

In [ ]:
met_names = remove_vars(vars_to_remove, met_names, df_met_class)

# Step 2: Metabolites selection by PLS-DA 

In [388]:
df_imputted = df_ctrl_cv[met_names + [y_var_cat]]

df_train_class_0 = df_imputted[df_imputted[y_var_cat]==0]
df_train_class_1 = df_imputted[df_imputted[y_var_cat]==1]

df_train_class_0 = transform_per_class(df_train_class_0)
df_train_class_1 = transform_per_class(df_train_class_1)

train_df = pd.concat([df_train_class_0, df_train_class_1])

df_imputted = train_df.sample(frac=1, random_state=SEED_GEN).reset_index(drop=True)

In [ ]:
%%R -i df_imputted -o df_pls_vip_out

library(Metrics)
library(ggplot2)
library(ggrepel)
library(mdatools)

vip_thres = 1.2
coef_thres = 0

dat_read <- df_imputted

set.seed(1000)

X = dat_read[,3:ncol(dat_read)-1]

y = dat_read[,ncol(dat_read)]
y = as.character(y)

set.seed(1000)
model <- plsda(cv=5, X, y, ncomp=10, center=TRUE, scale=TRUE,  
               ncomp.selcrit="wold")
#https://mda.tools/docs/mdatools-tutorial.pdf

summary(model)

predicted <- predict(model, X)$c.pred[,model$ncomp.selected, 2]
predicted[predicted < .1] <- 0
print(auc(y, predicted))

summary(model$calres, nc = 1)

plot(model)

plotPerformance(model)

plotYVariance(model)
plotYCumVariance(model)

plotXVariance(model)
plotXCumVariance(model)

set.seed(1000)
plotVIPScores(model, type = "h", show.labels = TRUE)

vip2_0 <- vipscores(model)
above_thres <- vip2_0[vip2_0[,1] >= vip_thres, ]
abv_thres<- colnames(X[row.names(above_thres)])


printList <- function(list) {
  for (item in 1:length(list)) {
    print(head(list[[item]]))
  }
}
printList(sort(abv_thres))
print(length(abv_thres))

vip2_0 <- data.frame(vip2_0[, 1])

colnames(vip2_0) <- c("VIP")
vip2_0["b"] <- rownames(vip2_0)

df_sorted <- vip2_0[order(vip2_0$VIP, decreasing = TRUE), ]

coef = getRegcoeffs(model, ny=2)
row.names.remove <- c("Intercept")
coef = coef[!(row.names(coef) %in% row.names.remove), ]
vip2_0["coef"] <- coef 


vip2_0["label"] = ""
vip2_0["label_plot"] = ""

# Loop through the rows of the data frame
for (i in 1:nrow(vip2_0)) {
  if ((vip2_0$VIP[i] > vip_thres) # & (vip2_0$coef[i] > coef_thres | vip2_0$coef[i] < coef_thres)
  ) {
    vip2_0$label[i] <- vip2_0$b[i]
    vip2_0$label_plot[i] <- strsplit(vip2_0$b[i], split = "@")[[1]][1]
  } else {
    vip2_0$label[i] <- ""
    vip2_0$label_plot[i] <- ""
  }
}

#create scatterplot with a label on every point
plot <- ggplot(vip2_0, aes(vip2_0$coef, vip2_0$VIP)) +
  geom_point() +
  geom_text_repel(aes(label = vip2_0$label_plot) #, max.overlaps = Inf
  )  

plot <- plot + theme_bw() + theme(panel.grid.major = element_blank(), 
                                  panel.grid.minor = element_blank()) 


plot <- plot + xlab("Coefficient") + ylab("VIP") 
plot <- plot + geom_hline(yintercept=vip_thres, linetype="dashed", color = "blue") 

plot

df_pls_vip_out <- vip2_0

In [ ]:
df_pls_vip = df_pls_vip_out[df_pls_vip_out['label']!=""]
df_pls_vip.index = list(df_pls_vip['label'])
df_pls_vip = df_pls_vip[["VIP", "coef"]]


df_pls_vip.columns = ['pls_vip', 'pls_coef']
df_pls_vip['model_2'] = 'PLS-DA'

pls_final = list(df_pls_vip.index)
pls_final = sorted(pls_final)

## checking correspondence between selected metabolites in external validation dataset

In [ ]:
# use bellow if you want to improve presentation of the metabolites names
pls_final_b = {}

for m in pls_final:
    m_b = # modifications
    pls_final_b[m] = m_b

# Step 3) Selected features analysis

In [396]:
df_met_code = pd.read_csv(# path of file with metabolomics superclasses, 
                                     # and the hmdb code
                                     , index_col=0)

In [ ]:
df_met_code['super_class'].value_counts()

In [ ]:
super_l = {}
sub_l = {}
for super_sub, n in df_met_code[['super_class', 'met_class']].value_counts().items():
    try:
        super_l[super_sub[0]].append([super_sub[1], n])
    except:
        super_l[super_sub[0]] = []
        super_l[super_sub[0]].append([super_sub[1], n])

super_l

In [ ]:
super_l = {}
sub_l = {}
for super_sub, n in df_met_code[['super_class', 'sub_class']].value_counts().items():
    try:
        super_l[super_sub[0]].append([super_sub[1], n])
    except:
        super_l[super_sub[0]] = []
        super_l[super_sub[0]].append([super_sub[1], n])

#dict(sorted(super_l.items(), key=lambda item: item[1], reverse=True))
super_l

In [ ]:
net_color_map = {'Lipids and lipid-like molecules': "black", 
                 'Organic acids and derivatives': "forestgreen", 
                 'Organic oxygen compounds': "sienna", 
                 'Organoheterocyclic compounds': "hotpink", 
                 'Organic nitrogen compounds': "grey",
                 'Nucleosides, nucleotides, and analogues': "#025a7a", 
                 'Homogeneous non-metal compounds':  "#480778",
                 'Alkaloids and derivatives': "red",
                 }

df_met_code['lab_col'] = np.nan

for i in df_met_code.index:
    df_met_code.loc[i, 'lab_col'] = net_color_map[df_met_code.loc[i, 'super_class']]
    
df_met_code['metab_b'] = np.nan
for i in df_met_code.index:
    m = df_met_code.loc[i, "Analyte"]
    m_b = # same as the pls_final_b
    df_met_code.loc[i, 'metab_b'] = m_b

col_code = dict(zip(df_met_code['metab_b'], df_met_code['lab_col']))

df_met_code.index = list(df_met_code['Analyte'])
lipids_vars = list(df_met_code[df_met_code['super_class']=="Lipids and lipid-like molecules"].index)
orgacid_vars = list(df_met_code[df_met_code['super_class']=="Organic acids and derivatives"].index)
orgoxy_vars = list(df_met_code[df_met_code['super_class']=="Organic oxygen compounds"].index)
orghetero_vars = list(df_met_code[df_met_code['super_class']=="Organoheterocyclic compounds"].index)
orgnitro_vars = list(df_met_code[df_met_code['super_class']=="Organic nitrogen compounds"].index)
nucleo_vars = list(df_met_code[df_met_code['super_class']=="Nucleosides, nucleotides, and analogues"].index)
homnonmetal_vars = list(df_met_code[df_met_code['super_class']=="Homogeneous non-metal compounds"].index)
alkaloids_vars = list(df_met_code[df_met_code['super_class']=="Alkaloids and derivatives"].index)
other_categories = orghetero_vars + orgnitro_vars + nucleo_vars + homnonmetal_vars + alkaloids_vars

## Proportion

In [ ]:
pie_labels = []
pie_values = []
pie_colors = []

for i in df_met_code['super_class'].value_counts().items():
    lab = i[0]
    val = i[1]
    col = net_color_map[i[0]]
    pie_labels.append(lab)
    pie_values.append(val)
    pie_colors.append(col)    
    
explode = [0.1] * len(pie_values) 

def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return f"{absolute:d} ({pct:.0f}%)"

plt.figure()
# plotting data on chart 
wedges, texts, labj = plt.pie(pie_values, labels=pie_labels, colors=pie_colors, 
        autopct= lambda pct: func(pct, pie_values), 
        textprops={'color': 'w', 'weight':'bold','size':'15'}, 
        pctdistance=0.81, radius=1.8,  labeldistance=1.1) 
  
for text, color in zip(texts, pie_colors):
    text.set_color(color)
    text.set_size(15)

plt.show()

# Steps 4 and 5: Classification model with XGBoost and Explanability by SHAP with metabolites categories

## With all selected features

In [ ]:
selected_imputer = 'mice'
selected_model = 'xgb'

settings_dict_xgb = tunner(df_ctrl_cv, pls_final, y_var_cat, selected_imputer, 
                           selected_model, SEED_GEN, per_class_imputation=True)
      
shap_viz_test(df_ctrl_cv, pls_final,  y_var_cat, selected_imputer, 
             selected_model, settings_dict_xgb, pls_final_b, 
             per_class_imputation=True, col_code=col_code)

## Models per class

In [ ]:
settings_dict_xgb = tunner(df_ctrl_cv, lipids_vars, y_var_cat, selected_imputer, 
                           selected_model, SEED_GEN, per_class_imputation=True)
shap_viz_test(df_ctrl_cv, lipids_vars, y_var_cat, selected_imputer, 
              selected_model, settings_dict_xgb, pls_final_b, 
              per_class_imputation=True, limit_show=True, col_code=col_code)

In [ ]:
settings_dict_xgb = tunner(df_ctrl_cv, orgacid_vars, y_var_cat, selected_imputer, 
                           selected_model, SEED_GEN, per_class_imputation=True)
shap_viz_test(df_ctrl_cv, orgacid_vars, y_var_cat, selected_imputer, 
              selected_model, settings_dict_xgb, pls_final_b, 
              per_class_imputation=True, limit_show=True, col_code=col_code)

In [ ]:
settings_dict_xgb = tunner(df_ctrl_cv, orgoxy_vars, y_var_cat, selected_imputer, 
                           selected_model, SEED_GEN, per_class_imputation=True)
shap_viz_test(df_ctrl_cv, orgoxy_vars, y_var_cat, selected_imputer, 
              selected_model, settings_dict_xgb, pls_final_b, 
              per_class_imputation=True, limit_show=True, col_code=col_code)

In [ ]:
settings_dict_xgb = tunner(df_ctrl_cv, other_categories, y_var_cat, selected_imputer, 
                           selected_model, SEED_GEN, per_class_imputation=True)
shap_viz_test(df_ctrl_cv, other_categories, y_var_cat, selected_imputer, 
              selected_model, settings_dict_xgb, pls_final_b, 
              per_class_imputation=True, limit_show=True, col_code=col_code)